🤫 підглядав тут 🙃 https://github.com/BogdanJeN/Geo/blob/main/Lab_06/lab_06.ipynb

In [1]:
import geopandas
import rasterio
from shapely.geometry import mapping
import json

In [2]:
clay_data = rasterio.open('../images_data/lab_06/soil_data/clay.tif')
sand_data = rasterio.open('../images_data/lab_06/soil_data/sand.tif')
density_data = rasterio.open('../images_data/lab_06//soil_data/density.tif')
moisture_data = rasterio.open('../images_data/lab_06/soil_moisture.tif')
fields_data = geopandas.read_file("../images_data/lab_06/field_centroids.geojson")

In [3]:
field_list = []

for index, row in fields_data.iterrows():
    id = row['id']
    name = row['Name']
    centroid = row['geometry'].centroid

    lon, lat = centroid.x, centroid.y

    row, col = moisture_data.index(lon, lat)
    clay = clay_data.read(1)[row, col]
    sand = sand_data.read(1)[row, col]
    density = density_data.read(1)[row, col]

    moisture = moisture_data.read(1)[row, col]

    features = {
        'type': 'Feature',
        'properties': {
            'id': id,
            'name': name,
            'soil': {
                'clay': float(clay),
                'sand': float(sand),
                'density': int(density)
            },
            'soil_moisture': int(moisture),
            'coordinates': {
                'lat': float(lat),
                'lng': float(lon)
            }
        },
        'geometry': mapping(centroid)
    }

    field_list.append(features)


In [5]:
feature_collection = {
    'type': 'FeatureCollection',
    'features': field_list
}

with open('../images_data/lab_06/field_data.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=4)